<a href="https://colab.research.google.com/github/jaunty0216/ForSophomoreCourse/blob/main/%E7%AC%AC%E5%8D%81%E9%80%B1_%E5%9C%B0%E7%8D%84%E7%BE%8E%E9%A3%9F%E5%AE%B6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 目標
在網路上尋找留言，並從留言內部擷取出特定的實體資訊，並加以分析

reference[連結文字](https://www.youtube.com/watch?v=urRaYxqq2CI)

# 大綱


*   匯入資料
*   讀取資料
*   處理資料
*   統計資料
*   視覺化資料
*   請Gemini分析資料
*   結論




# 開工

## 匯入資料

In [4]:
from google.colab import files
uploaded = files.upload()

Saving comments.csv to comments.csv


## 讀取資料

In [6]:
#知識點一
#前面的uploaded是dict，key=filename,value=filecontent，items()把這dict分開，
for filename,_ in uploaded.items():
    filepath = f"/content/{filename}"
    print(f"Filepath: {filepath}")

Filepath: /content/comments.csv


In [7]:
import pandas as pd
currentdata = pd.read_csv(filepath)
currentdata.head()

,comments
0,這間 Inferno Grill 的地獄火牛肉堡真是嚇死人，多汁、炙熱、熔岩般滑順！這是我人...
1,我原以為我對辣毫無感覺，直到我吃了 Inferno Grill 的魔焰雞翅。形容詞？瘋狂、黏...
2,骨湯小町 的怨靈豚骨拉麵，濃郁到像煮了千年的詛咒。每一口都像喝下祖先的懺悔——太香了我不後悔。
3,他們的岩漿煎餃超酥！地獄油香滲進每一層麵皮，骨湯小町 真是餃子界的黑暗女王。
4,我點了詛咒胡蘿蔔塔吉鍋。沒想到胡蘿蔔可以這麼狂暴香辣。辛焰集市根本是香料煉獄的餐桌體現。


## 處理資料

In [8]:
# 先把讀取到的csv檔案轉換成dataframe吧
import pandas as pd

currentdata_dataframe = pd.DataFrame(currentdata)

In [9]:
#匯入分析工具jieba
!pip install jieba

In [10]:
#斷詞，並儲存結果到新增的'tokenized'那一行
import jieba

def cutting(text):

  #新增字典
  words_to_add = ['詛咒胡蘿蔔','地獄火牛肉堡','魔焰雞翅','地獄三辣玉米餅']
  for word in words_to_add:
    jieba.add_word(word)

  #主要的斷詞功能
  generator = jieba.cut(text)
  tokens = list(generator)

  #把不要的字挑掉
  stopwords = [
    '的', '是', '在', '我', '有', '和', '了', '啊', '呀', '呢', '個', '會', '可以',
    '也', '都', '很', '非常', '比較', '這', '那', '一個', '一些', '什麼', '怎麼',
    '去', '來', '吃', '喝', '說', '看', '想', '覺得', '就是', '不是', '沒有', '還有',
    '你','我','他','你們','我們','他們',',','，','。','它','它們','為','甚麼','？','！',' ','誰',
    '回到','這','這麼','而','原以','竟然','人','沒','想到'
  ]

  filtered_content = [word for word in tokens if word not in stopwords]
  return filtered_content

currentdata['tokenized'] = currentdata['comments'].apply(cutting)
currentdata['tokenized']

,tokenized
0,"[這間, Inferno, Grill, 地獄火牛肉堡, 真是, 嚇死, 多汁, 、, 炙熱..."
1,"[對, 辣, 毫無感覺, 直到, Inferno, Grill, 魔焰雞翅, 形容, 詞, ..."
2,"[骨湯, 小, 町, 怨, 靈豚, 骨拉, 麵, 濃郁, 到, 像, 煮, 千年, 詛咒, ..."
3,"[岩漿, 煎, 餃超, 酥, 地獄, 油香, 滲進, 每, 一層, 麵, 皮, 骨湯, 小,..."
4,"[我點, 詛咒胡蘿蔔, 塔吉, 鍋, 胡蘿, 蔔, 狂暴, 香辣, 辛焰, 集市, 根本, ..."
5,"[靈食, 街尾, 地獄三辣玉米餅, 一, 咬, 下去, 就, 聽, 到, 自己, 靈魂, 拉..."
6,"[能, 亡靈, 仙人掌, 濃湯會, 絲滑, 濃烈, 靈食, 街尾給, 一碗, 詩意, 辣, ..."
7,"[再次, Inferno, Grill, 這次, 魔焰雞翅, 結果, 舌頭, 差點, 燃起,..."
8,"[怨, 靈豚, 骨拉, 麵, 哭, 因為, 辣, 因為, 讓, 想起, 地面, 上, 味道,..."


## 統計資料
利用gemini幫忙分析與歸類

In [11]:
# 引入gemini幫忙分析
from google.colab import userdata
GOOGLE_API_KKY = userdata.get("GOOGLE_API_KEY")

import google.generativeai as genai
genai.configure(api_key=GOOGLE_API_KKY)
model = genai.GenerativeModel("gemini-1.5-flash")

In [12]:
#取得回傳的array
#這個地方要先得到關鍵字的array，並用類似停用字的技巧，找出餐廳
#定義分析程式，回傳array給我
def analyze_tokens_for_entities(tokens):
  prompt_1 = f"根據內容，從文本中分析出餐廳的名稱，並把這些餐廳名稱直接以array的方式儲存起來並輸出，文本內容如下：{tokens}"
  response_1 = model.generate_content(prompt_1)
  #return response_1.text

  prompt_2 = f"根據內容，從文本中分析出料理的名字，並把這些料理名稱直接以array的方式儲存起來並輸出，文本內容如下：{tokens}"
  response_2 = model.generate_content(prompt_2)
  #return response_2.text

  a = response_1.text
  b = response_2.text

  return a,b

first_array,second_array = analyze_tokens_for_entities(currentdata['tokenized'])

In [13]:
# 開始分析
import jieba

def analize(text):

  #新增字典
  words_to_add = ['詛咒胡蘿蔔塔吉鍋','地獄火牛肉堡','魔焰雞翅','地獄三辣玉米餅','怨靈豚骨拉麵','岩漿煎地獄麵'
  ,'亡靈仙人掌','靈食街尾','骨湯小町','Inferno Grill','辛焰集市']
  for word in words_to_add:
    jieba.add_word(word)

  #主要的斷詞功能
  generator = jieba.cut(text)
  tokens = list(generator)

  #把不要的字挑掉
  stopwords = [
    '的', '是', '在', '我', '有', '和', '了', '啊', '呀', '呢', '個', '會', '可以',
    '也', '都', '很', '非常', '比較', '這', '那', '一個', '一些', '什麼', '怎麼',
    '去', '來', '吃', '喝', '說', '看', '想', '覺得', '就是', '不是', '沒有', '還有',
    '你','我','他','你們','我們','他們',',','，','。','它','它們','為','甚麼','？','！',' ','誰',
    '回到','這','這麼','而','原以','竟然','人','沒','想到','因為','所以','結果','詩意','堡','辣',
    '胡蘿蔔','火','詛咒'
  ]

  filtered_content_1 = [word for word in tokens if word not in stopwords]
  restaurant_name = [word for word in filtered_content_1 if word in first_array]

  filtered_content_2 = [word for word in tokens if word not in stopwords]
  food_name = [word for word in filtered_content_2 if word in second_array]

  return restaurant_name,food_name

#知識點三
currentdata['restaurant_name'] = currentdata['comments'].apply(lambda x: analize(x)[0])
currentdata['food_name'] = currentdata['comments'].apply(lambda x: analize(x)[1])

In [14]:
# 檢查輸出
currentdata[['restaurant_name','food_name']]

,restaurant_name,food_name
0,"[Inferno, Grill]",[地獄火牛肉堡]
1,"[Inferno, Grill]",[魔焰雞翅]
2,[骨湯小町],[怨靈豚骨拉麵]
3,[骨湯小町],"[岩漿, 煎, 地獄, 麵]"
4,[],"[詛咒胡蘿蔔塔吉鍋, 胡蘿, 蔔]"
5,[靈食街尾],[地獄三辣玉米餅]
6,[靈食街尾],[亡靈仙人掌]
7,"[Inferno, Grill]",[魔焰雞翅]
8,[骨湯小町],[怨靈豚骨拉麵]


## 視覺化資料

In [15]:
# 視覺化前的小準備
from collections import Counter
import pandas as pd # Import pandas if not already imported in this cell

# Flatten the list of lists in the 'tokenized' column
#知識點四
all_tokens_1 = [token for sublist in currentdata['restaurant_name'] for token in sublist]
all_tokens_2 = [token for sublist in currentdata['food_name'] for token in sublist]

# Now count the occurrences of each token
restaurant_count = Counter(all_tokens_1)
food_count = Counter(all_tokens_2)

print(restaurant_count)
print(food_count)

Counter({'Inferno': 3, 'Grill': 3, '骨湯小町': 3, '靈食街尾': 2})
Counter({'魔焰雞翅': 2, '怨靈豚骨拉麵': 2, '地獄火牛肉堡': 1, '岩漿': 1, '煎': 1, '地獄': 1, '麵': 1, '詛咒胡蘿蔔塔吉鍋': 1, '胡蘿': 1, '蔔': 1, '地獄三辣玉米餅': 1, '亡靈仙人掌': 1})


In [16]:
import plotly.express as px
import pandas as pd # Ensure pandas is imported

# 將 restaurant_count 轉換為 DataFrame
restaurant_df = pd.DataFrame.from_dict(restaurant_count, orient='index', columns=['count']).reset_index()
restaurant_df = restaurant_df.rename(columns={'index': 'Restaurant'})

# 使用 plotly express 繪製長條圖
fig = px.bar(restaurant_df, x='Restaurant', y='count', title='餐廳出現次數統計')

# 顯示圖形
fig.show()

In [17]:
import plotly.express as px
import pandas as pd

# 將 food_count 轉換為 DataFrame
#知識點五
food_df = pd.DataFrame.from_dict(food_count, orient='index', columns=['count']).reset_index()
food_df = food_df.rename(columns={'index': 'Food'})

# 使用 plotly express 繪製長條圖
fig = px.bar(food_df, x='Food', y='count', title='食物出現次數統計')

# 顯示圖形
fig.show()

# 反思與心得

心得：
從這一次的作業中，我更加掌握了如何使用jieba的技術，以及相關的技術分析

學到的相關概念會放在下面

# 一些小概念

In [20]:
#知識點一
#前面的uploaded是dict，key=filename,value=filecontent，items()把這dict分開，
for filename,_ in uploaded.items():
    filepath = f"/content/{filename}"
    print(f"Filepath: {filepath}")

Filepath: /content/comments.csv


In [24]:
# 知識點二
#jieba的斷詞邏輯與操作
import jieba

text = "我喜歡使用Google Colab進行數據分析。"
words_generator = jieba.cut(text)  # words_generator 是一個生成器
words_list = list(words_generator)  # words_list 是一個列表

# 生成器物件長這樣，這我不需要(so far)
# Building prefix dict from the default dictionary ...
# DEBUG:jieba:Building prefix dict from the default dictionary ...
# Dumping model to file cache /tmp/jieba.cache
# DEBUG:jieba:Dumping model to file cache /tmp/jieba.cache
# Loading model cost 1.152 seconds.
# DEBUG:jieba:Loading model cost 1.152 seconds.
# Prefix dict has been built successfully.
# DEBUG:jieba:Prefix dict has been built successfully.
# <generator object Tokenizer.cut at 0x79ca882e7480>

print(words_list)  # 輸出斷詞結果的列表

['我', '喜歡', '使用', 'Google', ' ', 'Colab', '進行', '數據', '分析', '。']


In [162]:
#知識點三
#對於會回傳多個list的函數，可以使用apply和lambda函數的結合來取得

def myfunction(x):

  first_stuff = []
  second_stuff = []
  third_stuff = []

  first_stuff.append(x**(1/2))
  second_stuff.append(x**2)
  third_stuff.append(x**3)

  return first_stuff,second_stuff,third_stuff

numbers = [1,2,3,4,5,6] #list

import pandas as pd
numbers_dataframe = pd.DataFrame(numbers,columns=['numbers'])
numbers_dataframe

numbers_dataframe['square_root'] = numbers_dataframe['numbers'].apply(lambda x: myfunction(x)[0])
numbers_dataframe['square'] = numbers_dataframe['numbers'].apply(lambda x: myfunction(x)[1])
numbers_dataframe['cube'] = numbers_dataframe['numbers'].apply(lambda x: myfunction(x)[2])

numbers_dataframe

,numbers,square_root,square,cube
0,1,[1.0],[1],[1]
1,2,[1.4142135623730951],[4],[8]
2,3,[1.7320508075688772],[9],[27]
3,4,[2.0],[16],[64]
4,5,[2.23606797749979],[25],[125]
5,6,[2.449489742783178],[36],[216]


知識點四

[expression for item1 in iterable1 for item2 in iterable2]

對應到

[token for sublist in currentdata['restaurant_name'] for token in sublist]

**其中：**

expression = token

item1 = sublist

iterable1 = currentdata['restaurant_name']

item2 = token

iterable2 = sublist


In [182]:
#知識點五
#list可以直接轉換為dataframe
#dict需要多一個'from_dict'的操作
#其中，把key換成index，需要加入"orient='index'"，並需要創建一個column來存放其value，不然預設的column標頭是0
my_list = (3,1,4,1,5,9,2,6)
import pandas as pd
my_list_dataframe = pd.DataFrame(my_list)
print(my_list_dataframe)

my_dict = {1:'my_word1',2:'my_word2',3:'my_word3',4:'my_word4',5:'my_word5',6:'my_word6',7:'my_word7',8:'my_word8'}
my_dict_dataframe = pd.DataFrame.from_dict(my_dict,orient='index',columns=['test_words'])
print(my_dict_dataframe)

   0
0  3
1  1
2  4
3  1
4  5
5  9
6  2
7  6
  test_words
1   my_word1
2   my_word2
3   my_word3
4   my_word4
5   my_word5
6   my_word6
7   my_word7
8   my_word8
